In [ ]:
import matplotlib.pyplot as plt
import blur

# Call the parabole_func method to fill the self.desired_fwhm list
ba = blur.blurCube()

ba.get_cube()

ba.read_cube()

ba.parabole_func()

# Create a list of band indices
band_indices = list(range(ba.wavelengths))

# Plot the desired FWHM values
plt.plot(band_indices, ba.desired_fwhm)

# Add labels and title
plt.xlabel('Band Index')
plt.ylabel('FWHM Spatial pixels')
plt.title('Spatial FWHM')

# Display the plot
plt.show()


In [ ]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import gridspec

def normalize_image(image):
    min_val = np.min(image)
    max_val = np.max(image)
    return (image - min_val) / (max_val - min_val)


def combine_patches_blured(parent_folder, key):
    patches = []
    for folder_name in os.listdir(parent_folder):
        image_folder = os.path.join(parent_folder, folder_name)
        for file in os.listdir(image_folder):
            file_path = os.path.join(image_folder, file)
            if file_path.endswith('.mat'):
                patch = sio.loadmat(file_path)[key]
                patch_rgb = patch[:, :, [33, 18, 2]]
                patches.append(patch_rgb)

    patches = np.array(patches).reshape((11, 3, *patch_rgb.shape))
    image = np.concatenate(np.concatenate(patches, axis=1), axis=1)
    image = normalize_image(image)

    return image

def combine_patches(image_paths, patch_key_prefix):
    patches = []

    data = sio.loadmat(image_paths)
    for i in range(1, 34):  # Assuming there are 33 patches
        patch_key = f"{patch_key_prefix}_{str(i).zfill(2)}_pred"
        if patch_key in data:
            patch = data[patch_key]
            patch_rgb = patch[:, :, [33, 18, 2]]  # Assuming these are the R, G, B channels
            patches.append(patch_rgb)

    patches = np.array(patches).reshape((11, 3, *patch_rgb.shape))
    image = np.concatenate(np.concatenate(patches, axis=1), axis=1)
    image = normalize_image(image)

    return image


def show(image, position, title):
    ax = plt.subplot(gs[position])
    ax.axis('off')
    ax.imshow(image)
    ax.set_title(title)

image_paths = '/home/kimado/master/DIP-HyperKite/Experiments/DHP_DARN/hico_dataset/N_modules_2/final_prediction.mat'
paths = '/home/kimado/master/DIP-HyperKite/datasets/hico/edge_image.nc'
image = combine_patches(image_paths, 'edge_image.nc')
ref = combine_patches_blured(paths, 'ref')
blurred = combine_patches_blured(paths, 'blurred')

fig = plt.figure(figsize=(5, 15))  # Adjust the size as needed
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], wspace=0.01)

show(image, 2, 'Fused image')
show(ref, 0, 'Reference')
show(blurred, 1, 'Blurred')

#plt.savefig('/home/kimado/master/HYPSO-1/fig/edge.pdf', format='pdf', bbox_inches='tight')  # Save the figure as a PDF
plt.show()


In [ ]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

def normalize_image(image):
    min_val = np.min(image)
    max_val = np.max(image)
    return (image - min_val) / (max_val - min_val)


def visualize_cube(blurred_cube_path,fusion_string):
    """
    fusion_result = sio.loadmat(fusion_result_path)
    image = fusion_result['H2010086172605.L1B_ISS.nc_01_pred'].transpose()
    print(fusion_result.keys())
    """
    blurred_image = sio.loadmat(blurred_cube_path)['blurred']
    image = sio.loadmat(blurred_cube_path)['ref']
    sharpest_image = sio.loadmat(blurred_cube_path)['sharpest_image']
    final_fusion = sio.loadmat(fusion_result_path)[fusion_string]
    #max_value = np.max(blurred_image)
    #print(f"The maximum value in the blurred_image is {max_value}")
    blurred_image = normalize_image(blurred_image)
    image = normalize_image(image)
    sharpest_image = normalize_image(sharpest_image)
    final_fusion = normalize_image(final_fusion)
    #qnr_values = qnr(sharpest_image, blurred_image, final_fusion)
    #print("QNR Value 1: ", qnr_values[0])
    #print("Lambda: ", qnr_values[1])
    #print("D_sr: ", qnr_values[2])
        
    R_f = image[:, :, 42-9]
    G_f = image[:, :, 27-9]
    B_f = image[:, :, 11-9]
    
    R = blurred_image[:, :, 42-9]
    G = blurred_image[:, :, 27-9]
    B = blurred_image[:, :, 11-9]

    Fus_r = final_fusion[:, :, 42-9]
    Fus_g = final_fusion[:, :, 27-9]
    Fus_b = final_fusion[:, :, 11-9]
    
    R_f = (R_f - R_f.min()) / (R_f.max() - R_f.min())
    G_f = (G_f - G_f.min()) / (G_f.max() - G_f.min())
    B_f = (B_f - B_f.min()) / (B_f.max() - B_f.min())
    
    R = (R - R.min()) / (R.max() - R.min())
    G = (G - G.min()) / (G.max() - G.min())
    B = (B - B.min()) / (B.max() - B.min())

    Fus_r = (Fus_r - Fus_r.min()) / (Fus_r.max() - Fus_r.min())
    Fus_g = (Fus_g - Fus_g.min()) / (Fus_g.max() - Fus_g.min())
    Fus_b = (Fus_b - Fus_b.min()) / (Fus_b.max() - Fus_b.min())


    fus_image = np.dstack((Fus_r, Fus_g, Fus_b))
    reference_image = np.dstack((R_f, G_f, B_f))
    blurred_image_stack = np.dstack((R, G, B))

    # Create a figure with 2 subplots
    plt.figure(figsize=(15, 5))

    # Display the rgb_image in the first subplot
    plt.subplot(1, 3, 1)
    plt.imshow(reference_image)
    plt.title('Reference patch')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(blurred_image_stack)
    plt.title('Blurred patch')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(fus_image)
    plt.title('Fused patch')
    plt.axis('off')

    plt.subplots_adjust(wspace=0.01, hspace=0.01)
    
    # New code to plot histograms
    #plt.figure(figsize=(15, 5))

    """
    # Plot the histogram of the reference image
    plt.subplot(1, 3, 1)
    plt.hist(reference_image.ravel(), bins=256, color='red', alpha=0.5)
    plt.title('Histogram of Reference Image')

    # Plot the histogram of the blurred image
    plt.subplot(1, 3, 2)
    plt.hist(blurred_image_stack.ravel(), bins=256, color='green', alpha=0.5)
    plt.title('Histogram of Blurred Image')

    # Plot the histogram of the fusion result
    plt.subplot(1, 3, 3)
    plt.hist(fus_image.ravel(), bins=256, color='blue', alpha=0.5)
    plt.title('Histogram of Fusion Result')
    """

    plt.show()
filename = 'H2010026150016.L1B_ISS.nc_11'
path = os.path.join('/home/kimado/master/DIP-HyperKite/datasets/hico', filename[:-3], filename, filename + '.mat')
fusion_result_path = '/home/kimado/master/DIP-HyperKite/Experiments/DHP_DARN/hico_dataset/N_modules_2/final_prediction.mat'
fus_string = 'H2010026150016.L1B_ISS.nc_11_pred'
visualize_cube(path, fus_string)